<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-atomistic-model" data-toc-modified-id="Generate-atomistic-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate atomistic model</a></span></li></ul></div>

## Generate atomistic model

In [1]:
import open3SPN2

# Initialize the DNA from a sequence.
# DNA type can be changed to 'A' or 'B'

seq='ATACAAAGGTGCGAGGTTTCTATGCTCCCACG'
dna=open3SPN2.DNA.fromSequence(seq, dna_type='B_curved',output_pdb=f'initial.pdb')

# Compute the topology for the DNA structure.
# Since the dna was generated from the sequence using X3DNA,
# it is not necesary to recompute the geometry.

dna.computeTopology(template_from_X3DNA=False)

# Create the system.
# To set periodic boundary conditions (periodicBox=[50,50,50]).
# The periodic box size is in nanometers.

dna.periodic=False
s=open3SPN2.System(dna, periodicBox=None)

#Add 3SPN2 forces

s.add3SPN2forces(verbose=True)


Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics


In [2]:
import sys
import simtk.openmm
import simtk.openmm.app
import simtk.unit
import numpy as np

#Initialize Molecular Dynamics simulations

simulation_platform='OpenCL' #['Reference', 'CPU', 'OpenCL', 'CUDA']
temperature=300 * simtk.unit.kelvin
s.initializeMD(temperature=temperature,platform_name=simulation_platform)
simulation=s.simulation

#Set initial positions

simulation.context.setPositions(s.coord.getPositions())

In [3]:
#Obtain total energy

energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

#Obtain detailed energy

energies = {}
for force_name, force in s.forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in s.forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

TotalEnergy -2046.579102 kJ/mol
Bond 2.1e-05 kJ/mol
Angle 0.001745 kJ/mol
Stacking -596.408569 kJ/mol
Dihedral -839.999756 kJ/mol
BasePair -518.252075 kJ/mol
CrossStacking -135.335464 kJ/mol
Exclusion 0.11901 kJ/mol
Electrostatics 43.296059 kJ/mol


In [4]:
#Add simulation reporters

dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 1000)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(energy_reporter)

In [5]:
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(10000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,-1641.8271484375,261.4463032229071
2000,3.999999999999781,-1633.8311767578125,330.8581715495927
3000,5.999999999999561,-1613.5548095703125,278.61341996231823
4000,7.999999999999341,-1698.197998046875,298.68064782441184
5000,10.000000000000009,-1645.2584228515625,320.45041946026095
6000,12.000000000000677,-1618.119873046875,323.2342588146424
7000,14.000000000001345,-1644.9344482421875,288.5324521291894
8000,16.00000000000201,-1684.2000732421875,280.55785824458434
9000,18.000000000000902,-1702.306884765625,308.65647812546183
10000,19.999999999999794,-1641.7108154296875,299.90949307239345
